<a href="https://colab.research.google.com/github/Oluwadara-Adepoju/Womxn/blob/main/Womxn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,  mean_squared_error, f1_score, roc_auc_score

from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

In [4]:
!pip install catboost==0.22 --quiet

     |████████████████████████████████| 64.4MB 63kB/s 


In [5]:
import os, sys, gc, warnings, random
import numpy as np 
import pandas as pd 
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import lightgbm as lgb  
from catboost import CatBoostClassifier ,Pool

from sklearn.metrics import auc, classification_report, roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [6]:
class CFG :
  SEED = 42
  n_splits = 5
  catboost_params = {'learning_rate':0.05,'iterations':10000,'eval_metric':'AUC',
                      'use_best_model' :True,'verbose':100,'random_seed': 0,
                      'devices':'0:1',}#'task_type':"GPU",}

  lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                'n_estimators': 500,'sub_sample' : 0.7,'colsample_bytree' : 0.6,
                'seed': SEED,'silent':False,'early_stopping_rounds': 100,
               }
  remove_features = ['ID', 'gender', 'target']
  categ_features = ['race','dwelling','dwelling_type','province_code','metro_code','nationality','RTH','marital_st','Lang_inside','Lang_outside','Education','lw_work','lw_business','help_on_household','job_or_business','nature_of_work']
  TARGET_COL = 'target'

In [7]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)

In [11]:
train= pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/Train (2).csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Women/Test (1).csv")
vd = pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/VariableDefinitions.csv")
ss=pd.read_csv ( "/content/drive/My Drive/Colab Notebooks/Women/SampleSubmission (1).csv")

In [12]:
seed_everything(CFG.SEED)

In [20]:
train['gender'] = label.fit_transform(train['gender'])
train['race'] = label.fit_transform(train['race'])
train['dwelling'] = label.fit_transform(train['dwelling'])
train['dwelling_type'] = label.fit_transform(train['dwelling_type'])
train['province_code'] = label.fit_transform(train['province_code'])
train['metro_code'] = label.fit_transform(train['metro_code'])
train['nationality'] = label.fit_transform(train['nationality'])
train['RTH'] = label.fit_transform(train['RTH'])
train['marital_st'] = label.fit_transform(train['marital_st'])
train['Lang_inside'] = label.fit_transform(train['Lang_inside'])
train['Lang_outside'] = label.fit_transform(train['Lang_outside'])
train['Education'] = label.fit_transform(train['Education'])
train['lw_work'] = label.fit_transform(train['lw_work'])
train['lw_business'] = label.fit_transform(train['lw_business'])
train['help_on_household'] = label.fit_transform(train['help_on_household'])
train['job_or_business'] = label.fit_transform(train['job_or_business'])
train['nature_of_work'] = label.fit_transform(train['nature_of_work'])


In [22]:
test['gender'] = label.fit_transform(test['gender'])
test['race'] = label.fit_transform(test['race'])
test['dwelling'] = label.fit_transform(test['dwelling'])
test['dwelling_type'] = label.fit_transform(test['dwelling_type'])
test['province_code'] = label.fit_transform(test['province_code'])
test['metro_code'] = label.fit_transform(test['metro_code'])
test['nationality'] = label.fit_transform(test['nationality'])
test['RTH'] = label.fit_transform(test['RTH'])
test['marital_st'] = label.fit_transform(test['marital_st'])
test['Lang_inside'] = label.fit_transform(test['Lang_inside'])
test['Lang_outside'] = label.fit_transform(test['Lang_outside'])
test['Education'] = label.fit_transform(test['Education'])
test['lw_work'] = label.fit_transform(test['lw_work'])
test['lw_business'] = label.fit_transform(test['lw_business'])
test['help_on_household'] = label.fit_transform(test['help_on_household'])
test['job_or_business'] = label.fit_transform(test['job_or_business'])
test['nature_of_work'] = label.fit_transform(test['nature_of_work'])

In [23]:
features_columns = [col for col in train.columns if col not in CFG.remove_features]
len(features_columns)

18

In [14]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True, random_state=CFG.SEED)

X , y   = train[features_columns] , train[CFG.TARGET_COL]

oof_cat = np.zeros((train.shape[0],))
test['target'] = 0
cat_preds= []

for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print(50*'-')
    print('Fold:',fold_+1)
    X_train, y_train = X.iloc[trn_idx,:], y[trn_idx] 
    X_test, y_test = X.iloc[val_idx,:], y[val_idx] 
       
    estimator = CatBoostClassifier(**CFG.catboost_params)
    estimator.fit(Pool(X_train,y_train,cat_features = CFG.categ_features),
                  eval_set = Pool(X_test,y_test,cat_features = CFG.categ_features),
                  early_stopping_rounds=200)
    
    y_pred_val = estimator.predict_proba(X_test)[:,1]
    oof_cat[val_idx] = y_pred_val
    y_pred_test = estimator.predict_proba(test[features_columns])[:,1]
    cat_preds.append(y_pred_test)
    print(50*'-')
    print()
print('OOF score :',roc_auc_score(y, oof_cat))

--------------------------------------------------
Fold: 1
0:	test: 0.5934718	best: 0.5934718 (0)	total: 85.9ms	remaining: 14m 18s
100:	test: 0.6620402	best: 0.6634041 (80)	total: 2.17s	remaining: 3m 33s
200:	test: 0.6697719	best: 0.6717815 (191)	total: 5.01s	remaining: 4m 4s
300:	test: 0.6623211	best: 0.6717815 (191)	total: 8.2s	remaining: 4m 24s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6717815217
bestIteration = 191

Shrink model to first 192 iterations.
--------------------------------------------------

--------------------------------------------------
Fold: 2
0:	test: 0.5723372	best: 0.5723372 (0)	total: 22.6ms	remaining: 3m 45s
100:	test: 0.6737363	best: 0.6832410 (41)	total: 2.21s	remaining: 3m 37s
200:	test: 0.6780163	best: 0.6832410 (41)	total: 5.13s	remaining: 4m 10s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6832409908
bestIteration = 41

Shrink model to first 42 iterations.
----------------------------------------------

In [15]:
catboost_preds = np.mean(cat_preds,axis=0)

In [24]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True, random_state=CFG.SEED)

X , y   = train[features_columns] , train[CFG.TARGET_COL]

oof_lgb = np.zeros((train.shape[0],))
test['target'] = 0
lgb_preds = []

for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print(50*'-')
    print('Fold:',fold_+1)

    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 
        
    train_data = lgb.Dataset(tr_x, label=tr_y,categorical_feature=CFG.categ_features)
    valid_data = lgb.Dataset(vl_x, label=vl_y,categorical_feature=CFG.categ_features)

    estimator = lgb.train(CFG.lgb_params,train_data,valid_sets = [train_data,valid_data],verbose_eval = 100)
    
    y_pred_val = estimator.predict(vl_x,num_iteration=estimator.best_iteration)
    oof_lgb[val_idx] = y_pred_val
    
    y_pred_test = estimator.predict(test[features_columns],num_iteration=estimator.best_iteration)
    lgb_preds.append(y_pred_test)
    print(50*'-')

print('OOF score :',roc_auc_score(y, oof_lgb))

--------------------------------------------------
Fold: 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.980203	valid_1's auc: 0.655392
Early stopping, best iteration is:
[15]	training's auc: 0.841261	valid_1's auc: 0.678532
--------------------------------------------------
--------------------------------------------------
Fold: 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.978235	valid_1's auc: 0.665996
Early stopping, best iteration is:
[23]	training's auc: 0.869643	valid_1's auc: 0.675895
--------------------------------------------------
--------------------------------------------------
Fold: 3
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.979532	valid_1's auc: 0.567698
Early stopping, best iteration is:
[26]	training's auc: 0.878125	valid_1's auc: 0.608586
--------------------------------------------------
-------------------------------------------------

In [25]:
lightgbm_preds = np.mean(lgb_preds,axis=0)

In [26]:
test['target'] = lightgbm_preds
submission = test[['ID', 'target']]
submission.to_csv('Women-Solution_lgbm.csv',index = False)

In [36]:
test['target'] = catboost_preds*0.9 +  lightgbm_preds*0.1
submission = test[['ID', 'target']]
submission.to_csv('Women-Solution_cat_lgbm_4.csv',index = False)

In [37]:
from google.colab import files
files.download('Women-Solution_cat_lgbm_4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>